In [ ]:
import pandas as pd
import requests
import io
import numpy as np  
from datetime import date, timedelta
import re
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
#from github import Github
#import github
import torch
import torch.nn as nn
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader
# Import nn.functional
import torch.nn.functional as F
import torch.optim as optim
from typing import Union, Tuple
import os
import sys
import time
from collections import OrderedDict
from sklearn.preprocessing import MinMaxScaler
from statistics import mean
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import math
import random
import imageio
#from sklearn.metrics import mean_absolute_percentage_error
matplotlib.style.use('seaborn')
%matplotlib inline


In [ ]:
class MLP(nn.Module):
  def __init__(self, input_dim, layers,output_dim):
    super(MLP, self).__init__()
    self.input_dim=input_dim
    self.n_layers=layers
    self.output_dim=output_dim
    in_features=input_dim
    out_features=16
    layers=[]
    for l in range(self.n_layers):
        if l==(self.n_layers-1):
          layers.append(nn.Linear(in_features=in_features, out_features=self.output_dim))
        else:
          layers.append(nn.Linear(in_features=in_features, out_features=out_features))
        if l%2==0:
          layers.append(nn.Tanh())
        else:
          layers.append(nn.ELU(inplace=True))
        in_features=out_features
        out_features=int(out_features/2)
    self.body = nn.Sequential(*layers)
  def forward(self, x):
    b, n_steps, features = x.shape
    #print(b,n_steps, features)
    x = x.reshape([b,n_steps*features])
    return self.body(x)

In [ ]:
Shortlisted_States=['Maharashtra','Delhi','Uttar-Pradesh','Kerala','Tamil-Nadu']
results_mlp=[]
for state in Shortlisted_States:
  best_models=[]
  df=pd.read_csv("https://raw.githubusercontent.com/sureshkuc/Data-Science-in-Life-Science-Project/main/Indian-States-Covid19-Datasets/"+state+".csv", parse_dates=["Date"]).drop(columns =["Unnamed: 0"])
  df = df[df["Date"] > "2020-03-19"]
  df = df.set_index("Date")
  df = df[['Confirmed', 'Recovered', 'Deceased', 'New_Confirmerd', 'New_Deaths', 'New_Recovered']]
  #print(df.describe())

  time_step=[5,7,15,30]
  Number_of_feature=[1,2,3,4,5,6]
  multi_feature=True
  for n_f in Number_of_feature:
    for t_s in time_step:
      train_loader, test_loader = data_preparation(df, scaling_range=(0,1),time_step=t_s,number_feature=n_f, response_variable_index=0,data_split_ratio=0.8)
      for n_layers in range(1,3,1):
          print(state,'n_f',n_f,'t_s',t_s,'n_layers',n_layers,'Error',mae,rmse,r2s)
          max_epochs=20
          random.seed(42)
          torch.manual_seed(42)
          np.random.seed(42)
          #CNN model with L1 loss
          #best_model=Call_CNN_model(state,dataset=(train_loader, test_loader), lr=1e-2,criterion=nn.L1Loss(),max_epochs=max_epochs)
          fc_model = MLP(input_dim=n_f*t_s, layers=n_layers,output_dim=1)
          cuda=torch.cuda.is_available()
          if cuda:
            fc_model = fc_model.cuda()
          fc_optim = optim.SGD(fc_model.parameters(), lr=1e-2, momentum=0.9)
          #fc_optim = optim.Adam(fc_model.parameters(), lr=1e-3)
          train_losses,test_losses,best_model = fit(fc_model, fc_optim,nn.L1Loss(),(train_loader, test_loader), max_epochs=max_epochs,cuda=cuda)
          #print(f'\nTraining took {end-start}s!')
          #plot_loss(max_epochs,train_losses,test_losses,model_name='CNN for '+state)
          fc_model = MLP(input_dim=n_f*t_s, layers=n_layers,output_dim=1)
          fc_model.load_state_dict(best_model)
          fc_model.eval()
          test_x,test_y=test_loader
          predictions=fc_model(test_x)
          test_y=test_y.cpu().detach().numpy()
          predictions=predictions.cpu().detach().numpy()
          mae=mean_absolute_error(test_y,predictions)
          rmse=math.sqrt(mean_squared_error(test_y,predictions))
          #mape=mean_absolute_percentage_error(test_y,predictions)
          r2s=r2_score(test_y,predictions)
          results_mlp.append([state,n_f,t_s,n_layers,mae,rmse,r2s])
          #print(state,n_f,t_s,n_layers,mae,rmse,r2s)